In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../input/unsw-nb15/UNSW_NB15_training-set.csv')

In [ ]:
##Data cleaning
##3 Create a new DataFrame X you will use for cleaning. Store the labels in the dataframe Y.
X = df.copy()
Y = X.label

columnNames= df.columns
print('Column names: ')
print(columnNames)

#prnt(X.label==1)



In [ ]:
##4 remove labels bound to attack as well as the 'id' field to avoid biaised learning
attack=df.loc[(df.label==1)&(df.attack_cat!='Normal')]
#del attack['label']
#attack
#attack = attack.drop(['id'],axis=1)
#attack
X.drop(columns=['attack_cat', 'label', 'id'], inplace=True)

In [ ]:
##5 List string fields, and perform one-hot encoding
string_fields = X.select_dtypes('object').columns.values
X = pd.get_dummies(X, columns=string_fields)

df.columns
print(X)

In [ ]:
##Machine learning analysis
##6 Through train_test_split on X and Y, and the XGBClassifier
##6.1 Extract train and test dataframes for X and Y
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=1)
print("Train size = "+str(len(X_train))+" Test_size = "+str(len(X_test)))

In [ ]:
##6.2 train the model
from xgboost import XGBClassifier, plot_importance, plot_tree
from sklearn.metrics import roc_auc_score

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

In [ ]:
##6.3 Extract the AUPRC
from sklearn.metrics import average_precision_score
weights = (Y == 0).sum()/(1.0*(Y==1).sum())

probabilities = xgb_model.fit(X_train, y_train).predict_proba(X_test)
print('AUPRC = {}'.format(average_precision_score(y_test, \
                                                  probabilities[:,1])))

In [ ]:
##6.4 Plot the relative importance of fields through plot_importance, using importance_type 'gain','weight','cover'
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(111)

colours = plt.cm.Set1(np.linspace(9,1,9))

ax = plot_importance(xgb_model,height=1,color=colours,grid=False, \
                    show_values=False, importance_type='gain', ax=ax);
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
    
ax.set_xlabel('importance score',size=10);
ax.set_ylabel('features',size=10);
ax.set_yticklabels(ax.get_yticklabels(),size=12);
ax.set_title('Importance_type gain',size=20)

In [ ]:
fig1 = plt.figure(figsize = (10,10))
ax1 = fig1.add_subplot(111)

colours1 = plt.cm.Set1(np.linspace(0,1,9))
ax1 = plot_importance(xgb_model,height=1,color=colours1,grid=False, \
                    show_values=False, importance_type='cover', ax=ax1);
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
    
ax1.set_xlabel('importance score',size=10);
ax1.set_ylabel('features',size=10);
ax1.set_yticklabels(ax1.get_yticklabels(),size=12);
ax1.set_title('Importance_type cover',size=20)

In [ ]:
fig2 = plt.figure(figsize = (10,10))
ax2 = fig2.add_subplot(111)

colours2 = plt.cm.Set1(np.linspace(2,5,9))
ax2 = plot_importance(xgb_model,height=1,color=colours2,grid=False, \
                    show_values=False, importance_type='weight', ax=ax2);
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
    
ax2.set_xlabel('importance score',size=10);
ax2.set_ylabel('features',size=10);
ax2.set_yticklabels(ax2.get_yticklabels(),size=12);
ax2.set_title('Importance_type weight',size=20)

6.5 What do you conclude wrt. the security status of the target system?


In [ ]:
##Statistics Analysis
##7 Apply a Principal Component Analysis (PCA) to the dataset X, to extract independent parameters of the model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#import matplotlib.pyplot as plt

x = StandardScaler().fit_transform(X.copy())
pca = PCA(n_components=2)
pca_x = pca.fit_transform(x)

principalDf = pd.DataFrame(data = pca_x, columns = ['principal component 1', \
                                                    'principal component 2'])
finalDf = pd.concat([principalDf, Y], axis = 1)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0,1]
colors = ['r','b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf.label == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'],
              finalDf.loc[indicesToKeep, 'principal component 2'],
              c = color,
              s = 50)
    ax.legend(targets)
    ax.grid()
    
    print("Independant parameters = "+str(pca.explained_variance_))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(X.copy())
pca = PCA(n_components=2)
pca_x = pca.fit_transform(x)

principalDf = pd.DataFrame(data = pca_x, columns = ['principal component 1', \
                                                   'principal component 2'])
finalDf = pd.concat([principalDf, Y], axis=1)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal component 1', fontsize = 15)
ax.set_ylabel('Principal component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0, 1]
colors = ['r', 'b']
for target, color in zip(targets, colors):
    indicesToKeep = finalDf.label == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'],
              finalDf.loc[indicesToKeep, 'principal component 2'],
              c = color,
              s = 50)
    ax.legend(targets)
    ax.grid()
    


In [ ]:
print("Independent parameter = "+str(pca.explained_variance_))

In [ ]:
##10.1. Build the Precision-Recall (PRC) curve
# precision-recall curve and f1 for an imbalanced dataset
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot


model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

lr_probs = model.predict_proba(X_test)
lr_probs = lr_probs[:, 1]
yhat = model.predict(X_test)
lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
lr_f1, lr_auc = f1_score(y_test, yhat), auc(lr_recall, lr_precision)
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
no_skill = len(y_test[y_test==1]) / len(y_test)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
pyplot.legend()
pyplot.show()

In [ ]:
##10.2. Build the Receiver Operating Characteristic (ROC) curve
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import sklearn.metrics as metrics
from matplotlib import pyplot
ns_probs = [0 for _ in range(len(y_test))]
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)
lr_probs = model.predict_proba(X_test)
#lr_probs = probabilities
lr_probs = lr_probs[:, 1]
#lr_probs = probabilities[:, 1]
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
ns_fpr, ns_tpr, _ = metrics.roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = metrics.roc_curve(y_test, lr_probs)
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

In [ ]:
##Plot the learning curve
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from matplotlib import pyplot

yhat = model.predict(X_test)
score = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % score)
results = model.evals_result()
pyplot.plot(results['validation_0']['logloss'], label='train')
pyplot.plot(results['validation_1']['logloss'], label='test')
pyplot.legend()
pyplot.show()